Penalize absent features impacint predictions by:   
Absent protein in e.g. brain with SHAP 0.06 - the average SHAP of said protein in Brain training data e.g. 0.04 ==> reduces prediction to 0.02

In [1]:
import MLMarker_app
import pandas as pd
import numpy as np
from MLMarker_app import MLMarker
import matplotlib.pyplot as plt
import random
import seaborn as sns
import shap
import joblib

/home/compomics/miniconda3/envs/mlmarker/lib/python3.9/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [2]:
# Load training data
training_data_path = "/home/compomics/git/MLMarker/models/train_binary_TP_4000features_95to75missingness_2024.csv"
training_data = pd.read_csv(training_data_path)
#set the name of the last column to tissue_name
training_data.columns.values[-1] = 'tissue_name'

# Load model
model_path = "/home/compomics/git/MLMarker/models/binary_TP_4000features_95to75missingness_2024.joblib"
model = joblib.load(model_path)

# Load features
features_path = "/home/compomics/git/MLMarker/models/binary_features_TP_4000features_95to75missingness_2024.txt"
with open(features_path, 'r') as f:
    feature_names = f.read().split(',\n')

# Assuming the target (labels) column is named "target"
X_train = training_data[feature_names]
y_train = training_data['tissue_name']


In [3]:

test_sample = pd.read_csv('/home/compomics/git/MLMarker/Projects/organoids_pauline/tmp_test.csv')

In [6]:
test = MLMarker(test_sample, binary=True)
adj_shap_values = MLMarker.adjusted_shap_values_df(test, n_preds=100)
adj_shap_values.sum(axis=1).sort_values(ascending=False)

tissue
Testis             0.132226
Esophagus          0.089888
Prostate           0.074227
Retina             0.053439
Adipose tissue     0.028958
Placenta           0.026039
Lymph node         0.022739
Liver              0.018745
Gall bladder       0.018013
Skeletal muscle    0.017027
Small intestine    0.012060
Heart              0.005481
Urinary bladder    0.000563
Ureter            -0.003916
Adrenal gland     -0.014067
Brain             -0.015046
Appendix          -0.021837
Colon             -0.025902
Parotid gland     -0.036367
Tonsil            -0.039111
NK-cells          -0.043472
T-cells           -0.056445
Ovary             -0.072603
B-cells           -0.170642
dtype: float64

In [4]:
diseased = pd.read_csv('/home/compomics/git/MLMarker/data/diseased_atlas_20240403.csv')
diseased.head()


,index,assay_id,cell_type,tissue_name,disease_status,fluid,A0A024RBG1,A0A075B6H7,A0A075B6H8,A0A075B6H9,...,Q9Y6X8,Q9Y6X9,Q9Y6Y0,Q9Y6Y1,Q9Y6Y8,Q9Y6Y9,Q9Y6Z4,Q9Y6Z5,Q9Y6Z7,W5XKT8
0,24,2549,Brain,Brain,Diseased,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,25,2564,Brain,Brain,Diseased,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26,2838,Brain,Brain,Diseased,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,2992,Brain,Brain,Diseased,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,3230,Brain,Brain,Diseased,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
missing_columns = set(X_train.columns) - set(diseased.columns)
for column in missing_columns:
    diseased[column] = 0
diseased2 = diseased[X_train.columns.tolist() + ['tissue_name']]

In [6]:
X_train.iloc[5:6,:], y_train.iloc[5:6]

(   A0A075B6H7  A0A075B6H9  A0A075B6I0  A0A075B6I1  A0A075B6K5  A0A075B6Q5  \
 5         0.0         0.0         0.0         0.0         0.0         0.0   
 
    A0A075B6R2  A0A075B6S4  A0A075B6S5  A0A087WSY6  ...  Q9Y6V7  Q9Y6W3  \
 5         0.0         0.0         0.0         0.0  ...     0.0     0.0   
 
    Q9Y6W5  Q9Y6X0  Q9Y6X3  Q9Y6X5  Q9Y6X8  Q9Y6X9  Q9Y6Y0  Q9Y6Y8  
 5     1.0     0.0     0.0     0.0     0.0     0.0     0.0     1.0  
 
 [1 rows x 6877 columns],
 5    B-cells
 Name: tissue_name, dtype: object)

In [7]:
# Initialize SHAP TreeExplainer
explainer = shap.TreeExplainer(model)

# Compute SHAP values for all training samples
shap_values = explainer.shap_values(X_train.iloc[5:6,:])

# shap_values will be a list where each element corresponds to a class
# For binary classification, it will be shap_values[0] for class 0 and shap_values[1] for class 1


In [8]:
model.classes_

array(['Adipose tissue', 'Adrenal gland', 'Appendix', 'B-cells', 'Brain',
       'Colon', 'Esophagus', 'Gall bladder', 'Heart', 'Liver',
       'Lymph node', 'NK-cells', 'Ovary', 'Parotid gland', 'Placenta',
       'Prostate', 'Retina', 'Skeletal muscle', 'Small intestine',
       'T-cells', 'Testis', 'Tonsil', 'Ureter', 'Urinary bladder'],
      dtype=object)

In [9]:
import pandas as pd
import numpy as np

# Initialize an empty DataFrame to store the feature values for each class
feature_values = pd.DataFrame(columns=list(X_train.columns) + ['class'])

# Loop over all samples
for i in range(len(X_train)):
    # Predict a sample
    sample = X_train.iloc[i:i+1, :]
    prediction = model.predict(sample)

    # Compute SHAP values for the sample
    shap_values = explainer.shap_values(sample)

    # Convert SHAP values to DataFrame
    shap_df = pd.DataFrame(shap_values[0])

    # Transpose the DataFrame so that features are columns and classes are rows
    shap_df = shap_df.T

    # Take the row with the highest total
    highest_class = shap_df.sum(axis=1).idxmax()

    # Store the feature values for that class
    feature_values.loc[i, X_train.columns] = sample.values[0]
    feature_values.loc[i, 'class'] = highest_class

# Once done all samples, take the average of the feature values for each class
average_feature_values = feature_values.groupby('class').mean()

# Replace class numbers with model.classes_ corresponding class names
average_feature_values.index = model.classes_


KeyboardInterrupt: 

In [10]:
zero_sample = pd.DataFrame(np.zeros((1, len(diseased2.columns))), columns=diseased2.columns)
zero_test = MLMarker(zero_sample, binary = True)
zero_shaps = MLMarker.shap_values_df(zero_test, n_preds=100)
zero_shaps.sum(axis=1)

tissue
B-cells            0.333355
T-cells            0.065845
Ovary              0.064358
Parotid gland      0.024008
NK-cells           0.014795
Brain              0.011170
Tonsil             0.007617
Adrenal gland     -0.006624
Appendix          -0.009369
Colon             -0.011678
Ureter            -0.022121
Skeletal muscle   -0.027562
Heart             -0.031217
Urinary bladder   -0.033081
Placenta          -0.034176
Esophagus         -0.034232
Testis            -0.035545
Liver             -0.035566
Small intestine   -0.038405
Prostate          -0.038526
Adipose tissue    -0.040131
Lymph node        -0.040186
Gall bladder      -0.041061
Retina            -0.041667
dtype: float64

In [11]:
ones_sample = pd.DataFrame(np.ones((1, len(diseased2.columns))), columns=diseased2.columns)
ones_test = MLMarker(ones_sample, binary = True)
ones_shaps = MLMarker.shap_values_df(ones_test, n_preds=100)
ones_shaps.sum(axis=1)

tissue
Retina             0.286325
Skeletal muscle    0.094367
Prostate           0.022956
Liver              0.016333
Esophagus          0.016099
Parotid gland      0.010333
Adipose tissue     0.010139
Testis             0.009119
Small intestine   -0.010439
Heart             -0.012261
Gall bladder      -0.012710
Lymph node        -0.019515
Brain             -0.022542
Placenta          -0.024542
Adrenal gland     -0.025076
T-cells           -0.031083
NK-cells          -0.032700
Appendix          -0.037246
Colon             -0.037508
Ureter            -0.038275
Urinary bladder   -0.038497
Tonsil            -0.040484
B-cells           -0.041125
Ovary             -0.041667
dtype: float64

In [12]:
test = MLMarker(test_sample, binary=True)
#coumn names with zero values
missing_proteins = test_sample.columns[test_sample.iloc[0] == 0]
example_shap_values = MLMarker.shap_values_df(test, n_preds=100)
example_shap_values

,A0A075B6H7,A0A075B6H9,A0A075B6I0,A0A075B6I1,A0A075B6K5,A0A075B6Q5,A0A075B6R2,A0A075B6S4,A0A075B6S5,A0A087WSY6,...,Q9Y6V7,Q9Y6W3,Q9Y6W5,Q9Y6X0,Q9Y6X3,Q9Y6X5,Q9Y6X8,Q9Y6X9,Q9Y6Y0,Q9Y6Y8
tissue,,,,,,,,,,,,,,,,,,,,,
Testis,0.000131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Esophagus,-0.000109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Prostate,0.000071,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ureter,-0.000958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-cells,0.000047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NK-cells,0.000040,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tonsil,0.000061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.087727e-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Placenta,0.000031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Urinary bladder,0.000107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
absent_shap = example_shap_values[missing_proteins]
#absent and value not equal to zero
notcontributing = example_shap_values.loc[:, example_shap_values.sum() == 0] # 
absent_shap_and_contributing = absent_shap.loc[:, absent_shap.sum() != 0]
present_shap = example_shap_values.drop(missing_proteins, axis=1)
zero_absent_shap = zero_shaps[absent_shap_and_contributing.columns.tolist()]
penalty = absent_shap_and_contributing - (2*zero_absent_shap)
combined_df = pd.concat([present_shap, notcontributing, penalty], axis=1)
combined_df_sum = combined_df.sum(axis=1)
combined_df_sum_sorted = combined_df_sum.sort_values(ascending=False)
combined_df_sum_sorted

tissue
Testis             0.170929
Esophagus          0.124209
Prostate           0.096744
Retina             0.060951
Placenta           0.057719
Adipose tissue     0.054907
Lymph node         0.051861
Liver              0.046368
Gall bladder       0.041275
Small intestine    0.031621
Skeletal muscle    0.031605
Urinary bladder    0.031100
Heart              0.027932
Ureter             0.027800
Adrenal gland     -0.003993
Appendix          -0.009243
Tonsil            -0.009438
Colon             -0.012423
Brain             -0.019786
NK-cells          -0.026219
Parotid gland     -0.063445
T-cells           -0.102776
Ovary             -0.127259
B-cells           -0.480438
dtype: float64